GOAL: Present alignment skills
* BioPython
* Python
* MUSCLE
* algorithm development
* Computing a conservation score

[Large-Scale Sequence Analysis of Hemagglutinin of Influenza A Virus Identifies Conserved Regions Suitable for Targeting an Anti-Viral Response](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0009268)

They are aligning proteins, not RNA.

1. full length HA sequences download
2. seperate files for each H1, H2 etc
3. align each subtype using MUSCLE 3.6 - filtering out frameshifts and partials
4. each profile for each multiple alignment then aligned to other profiles using profile-profile aligner COMPASS
5. Compute a conservation score for each column of the final alignment

[Identification of potential conserved RNA secondary structure throughout influenza A coding regions](https://rnajournal.cshlp.org/content/17/6/991.full_)

1. Interesting that this paper aligns first based on protein, then RNA
2. six full influenza sets were downloaded from human avian swine
3. divide each by segment
4. coding regions translated with BioEdit
5. protein sequences aligned with ClustalW
6. aligned sequences turned back into nt, submitted to RNAz 2.0 for conserved structures

Now that I've downloaded all the records that I need and gotten an idea of what they hold, it's time to look at sequences in an attempt to answer question 3: which is how similiar were the recommended vaccine strains to the reported sequences during the 17-18 season. 

How do I compare the differences between the sequences? Will have to look into BioPython to see what in available in that library to compare genetics. 

First I'm going to attempt to download the vaccine candidate sequences from NCBI.

First one is A/Michigan/45/2015 (H1N1) pdm09-like virus

Retrieve that from IRD

In [7]:
# Import data analysis libraries 
import numpy as np
import pandas as pd

# Import visualization libraries
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('paper', font_scale=1.)
import matplotlib.pyplot as plt
%matplotlib inline 

In [8]:
#Load up the vaccine candidate sequences into a dataframe, first with H1N1

AM42_H1N1_db = pd.read_csv('A-michigan-45-2015.tsv', sep='\t')
AM42_H1N1_db.columns

Index(['Strain Name', 'Complete Genome', 'Subtype', 'Collection Date', 'Host',
       'Country', 'State/Province', 'Geographic Grouping', 'Flu Season',
       'Submission Date', 'Passage History', 'Specimen Source Health Status',
       '1 PB2', '2 PB1', '3 PA', '4 HA', '5 NP', '6 NA', '7 MP', '8 NS', 'Age',
       'Gender', 'M2 31N', 'M2 26F', 'M2 27A', 'M2 30T', 'M2 34E',
       'NA 275Y N1', 'NA 292K N2', 'NA 119V N2', 'NA 294S N2', 'PB1-F2 66S',
       'PB2 E627K', 'PB2 D701N', 'PB2 A199S', 'PB2 A661T', 'PB2 V667I',
       'PB2 K702R', 'PA S409N', 'NP L136M', 'M2 A16G', 'M2 C55F', 'NS1 T92E',
       'RERRRKKR', 'Sensitive Drug', 'Resistant Drug', 'Submission Date.1',
       'NCBI Taxon ID', 'pH1N1-like', 'US Swine H1 Clade',
       'Global Swine H1 Clade test', 'H5 Clade', 'Unnamed: 52'],
      dtype='object')

In [9]:
AM42_H1N1_db.head(3)

,Strain Name,Complete Genome,Subtype,Collection Date,Host,Country,State/Province,Geographic Grouping,Flu Season,Submission Date,...,RERRRKKR,Sensitive Drug,Resistant Drug,Submission Date.1,NCBI Taxon ID,pH1N1-like,US Swine H1 Clade,Global Swine H1 Clade test,H5 Clade,Unnamed: 52
0,A/Michigan/45/2015,Yes,H1N1,09/07/2015,Human,USA,Michigan,North America,-N/A-,2017-08-24,...,No,-N/A-,-N/A-,08/24/2017,1777792,Mixed Positive and Negative Segments,npdm,1A.3.3.2,-N/A-,NaN


OK, lets look at just the HA segment - column named '4 HA'

In [10]:
AM42_H1N1_db['4 HA']

0    KY117023,KY090610,KU933493,KU509703
Name: 4 HA, dtype: object

In [11]:
#going to put this into a list so can interate over and call into NCBI
#listing the contents of a column will yield a series, not a list so need to call .tolist() to convert

acc_list = AM42_H1N1_db['4 HA'].tolist()
acc_list = acc_list[0].split(',')
print(acc_list)

['KY117023', 'KY090610', 'KU933493', 'KU509703']


So this give me the accession numbers in a list. Going iterate over this list to call into NCBI and download these sequences and put this into a database.

In [12]:
# Using the Bio Entrez module, going to retreive the sequence data the four accession nos
# Then add this to a new dataframe for storage.

from Bio import Entrez

Entrez.email = "adriana@dranalytics.co"
df = pd.DataFrame(columns=['accession', 'FASTA'])


for i in range(len(acc_list)):
    FASTA = Entrez.efetch(db="nucleotide", id=acc_list[i], rettype="fasta", retmode="text").read()
    df.loc[i] = {'accession': acc_list[i], 'FASTA': FASTA}

df.head(6)


,accession,FASTA
0,KY117023,>KY117023.1 Influenza A virus (A/Michigan/45/2...
1,KY090610,>KY090610.1 Influenza A virus (A/Michigan/45/2...
2,KU933493,>KU933493.1 Influenza A virus (A/Michigan/45/2...
3,KU509703,>KU509703.1 Influenza A virus (A/Michigan/45/2...


Now going to attempt an alignment - understand how different are the strains at the sequence level
Difference between global (stretcher), local (blastn) , semi-global alignment (minimap2)

[Pairwise2](http://biopython.org/DIST/docs/api/Bio.pairwise2-module.html)

In [13]:
# Using pairwise2 module allows pairwise sequence alignment for global and local alignments

from Bio import pairwise2
from Bio.pairwise2 import format_alignment

seq1 = df.loc[0,'FASTA']
seq2 = df.loc[1,'FASTA']

alignments = pairwise2.align.globalxx(seq1, seq2)
print(format_alignment(*alignments[0]))

>KY1----17023.1 Influenza A virus (A/Michigan/45/2015(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
GGAAAAACAAAAGCAACAAAAATGAAGGCAATACTAGTAGTTCTGCTATATACATTTACAACCGCAAATG
CAGACACATTATGTATAGGTT-ATCATGCGAACAATTCAACAGACACTGTAGACACAGTACTAGAAAAGAA
TGTAACAGTAACACACTCTGT-TAACCTTCTGGAAGACAAGCATAACGGAAAACTATGCAAACTAAGAGGG
GTAGCCCCATTGCATTTGGGT-AAATGTAACATTGCTGGCTGGATCCTGGGAAATCCAGAGTGTGAATCAC
TCTCCACAGCAAGTTCATGGT-CCTACATTGTGGAAACATCTAATTCAGACAATGGAACGTGTTACCCAGG
AGATTTCATCAATTATGAGGA-GCTAAGAGAGCAATTGAGCTCAGTGTCATCATTTGAAAGGTTTGAGATA
TTCCCCAAGACAAGTTCATGG-CCCAATCATGACTCGAACAAAGGTGTAACGGCAGCATGTCCTCACGCTG
GAGCAAAAAGCTTCTACAAAA-ACTTGATATGGCTAGTTAAAAAAGGAAATTCATACCCAAAGCTTAACCA
ATCCTACATTAATGATAAAGG-GAAAGAAGTCCTCGTGCTGTGGGGCATTCACCATCCATCTACTACTGCT
GACCAACAAAGTCTCTATCAG-AATGCAGATGCATATGTTTTTGTGGGGACATCAAGATACAGCAAGAAGT
TCAAGCCGGAAATAGCAACAA-GACCCAAAGTGAGGGATCAAGAAGGGAGAATGAACTATTACTGGACACT
AGTAGAGCCGGGAGACAAAAT-AACATTCGAAGCAACTGGAAATCTAGTGGTACCGAGATATGCATTCACA
ATGGAAAGAAATGCTGGATCT-GGTATT

In [39]:
from Bio.Align.Applications import MuscleCommandline
from Bio.Data import CodonTable
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

my_seq = Seq('GGAAAAACAAAAGCAACAAAAATGAAGGCAATACTAGTAGTTCTGCTATATACATTTACAACCGCAAATG', IUPAC.unambiguous_dna)
my_seq



Seq('GGAAAAACAAAAGCAACAAAAATGAAGGCAATACTAGTAGTTCTGCTATATACA...ATG', IUPACUnambiguousDNA())

In [40]:
my_seq.complement()

Seq('CCTTTTTGTTTTCGTTGTTTTTACTTCCGTTATGATCATCAAGACGATATATGT...TAC', IUPACUnambiguousDNA())

In [42]:
my_seq.translate()

/Users/adrianaip/anaconda3/lib/python3.7/site-packages/Bio/Seq.py:2576: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


Seq('GKTKATKMKAILVVLLYTFTTAN', IUPACProtein())

In [ ]:
from iab.algorithms import progressive_msa, tree_from_distance_matrix
from functools import partial

f = 'run1_16s/rev_seqs/1AM1JR7QWMSFA.fastq'
seqs = [DNA(e) for e in itertools.islice(skbio.io.read(f, format='fastq', verify=False, variant='illumina1.8'), 10)]
seqs = [e for e in seqs if not e.has_degenerates()]
msa = progressive_msa(seqs, global_pairwise_align_nucleotide)
msa.write('msa10.fna')
msa = skbio.alignment.TabularMSA.read('msa10.fna', constructor=DNA)